<a href="https://colab.research.google.com/github/renannevesr/RACCOON.MONKS/blob/main/Desafio_Raccoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fsspec
import requests
import pandas as pd
import json
import numpy as np
from pandas.core.series import Series

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Carregamento de bases

In [2]:
link_ingressos ="https://us-central1-raccoon-bi.cloudfunctions.net/psel_de_ingressos"
link_compras = "https://us-central1-raccoon-bi.cloudfunctions.net/psel_de_compras"
link_shows = "https://us-central1-raccoon-bi.cloudfunctions.net/psel_de_shows"

In [3]:
response = requests.get(link_shows)
data = response.json()
df_shows_base = pd.DataFrame.from_dict(data)

In [4]:
response = requests.get(link_ingressos)
data = response.json()
df_ingressos_base = pd.DataFrame.from_dict(data)

In [5]:
df_compras_base = pd.read_csv(link_compras)

In [6]:
df_shows_base

,Nascent Letter,Symbolic Toy,Terrestrial Chair
ano,2020,2020,2020
dia,2,3,1
mes,1,1,1


In [7]:
df_ingressos_base

,ano,dia,mes,nome,status,tipo
0,2020,1,1,Benjamin Mcdonald,Nao Concluido,VIP
1,2020,1,1,Mitchell Kostyla,Nao Concluido,VIP
2,2020,1,1,Gerald Allen,Nao Concluido,VIP
3,2020,1,1,Frederick Bank,Problema no Pagamento,VIP
4,2020,1,1,Tina Yee,Nao Concluido,VIP
...,...,...,...,...,...,...
95,2020,3,1,Richard Jimenez,Nao Concluido,Pista Premium
96,2020,3,1,Curtis Hernandez,Problema no Pagamento,Pista Premium
97,2020,3,1,Alice Gamba,Concluido,Pista Premium
98,2020,3,1,Brenda Kozak,Problema no Pagamento,Pista Premium


In [8]:
df_compras_base

,nome,show,gastos
0,Joseph Wray,Symbolic Toy,3142.35
1,Barbara Mcneil,Symbolic Toy,2431.78
2,Eduardo Espenshade,Symbolic Toy,386.32
3,Lucia Hayes,Symbolic Toy,3373.49
4,Johnny Hairston,Symbolic Toy,2805.88
...,...,...,...
495,Cassandra Gonsalves,Terrestrial Chair,154.39
496,Christopher Mueller,Symbolic Toy,1797.44
497,Christopher Mueller,Nascent Letter,1511.91
498,Christopher Mueller,Nascent Letter,3439.69


#Tratamento das DF

Adição da coluna show com os parametros corretos

In [9]:
df_shows =df_shows_base.T
df_shows

,ano,dia,mes
Nascent Letter,2020,2,1
Symbolic Toy,2020,3,1
Terrestrial Chair,2020,1,1


In [10]:
df_shows['show'] =df_shows_base.T.index
df_shows

,ano,dia,mes,show
Nascent Letter,2020,2,1,Nascent Letter
Symbolic Toy,2020,3,1,Symbolic Toy
Terrestrial Chair,2020,1,1,Terrestrial Chair


Ordenamento alfabético para melhor visualização dos dados

In [11]:
df_ingressos= df_ingressos_base.sort_values(by=['nome'])
df_ingressos.reset_index(drop=True, inplace=True)
df_ingressos

,ano,dia,mes,nome,status,tipo
0,2020,3,1,Alice Gamba,Concluido,Pista Premium
1,2020,2,1,Alice Gamba,Nao Concluido,VIP
2,2020,2,1,Amy Mejia,Problema no Pagamento,Pista Premium
3,2020,1,1,Andrea Galyen,Nao Concluido,Pista
4,2020,3,1,Andrea Galyen,Nao Concluido,VIP
...,...,...,...,...,...,...
95,2020,1,1,Tina Yee,Nao Concluido,VIP
96,2020,1,1,Travis Whatley,Problema no Pagamento,Pista
97,2020,1,1,William Barnes,Nao Concluido,VIP
98,2020,3,1,Yong Baltazar,Nao Concluido,VIP


Adição e população de coluna show no DF ingressos

In [12]:
df_ingressos["show"] = " "

In [13]:
df_ingressos['show'] = df_ingressos['dia'].map(df_shows.set_index('dia')['show']).fillna(0)

In [14]:
df_ingressos

,ano,dia,mes,nome,status,tipo,show
0,2020,3,1,Alice Gamba,Concluido,Pista Premium,Symbolic Toy
1,2020,2,1,Alice Gamba,Nao Concluido,VIP,Nascent Letter
2,2020,2,1,Amy Mejia,Problema no Pagamento,Pista Premium,Nascent Letter
3,2020,1,1,Andrea Galyen,Nao Concluido,Pista,Terrestrial Chair
4,2020,3,1,Andrea Galyen,Nao Concluido,VIP,Symbolic Toy
...,...,...,...,...,...,...,...
95,2020,1,1,Tina Yee,Nao Concluido,VIP,Terrestrial Chair
96,2020,1,1,Travis Whatley,Problema no Pagamento,Pista,Terrestrial Chair
97,2020,1,1,William Barnes,Nao Concluido,VIP,Terrestrial Chair
98,2020,3,1,Yong Baltazar,Nao Concluido,VIP,Symbolic Toy


Adicação e população da coluna sobre o dia dos shows.

In [15]:
df_compras_base["dia"] = "0"
df_compras_base

,nome,show,gastos,dia
0,Joseph Wray,Symbolic Toy,3142.35,0
1,Barbara Mcneil,Symbolic Toy,2431.78,0
2,Eduardo Espenshade,Symbolic Toy,386.32,0
3,Lucia Hayes,Symbolic Toy,3373.49,0
4,Johnny Hairston,Symbolic Toy,2805.88,0
...,...,...,...,...
495,Cassandra Gonsalves,Terrestrial Chair,154.39,0
496,Christopher Mueller,Symbolic Toy,1797.44,0
497,Christopher Mueller,Nascent Letter,1511.91,0
498,Christopher Mueller,Nascent Letter,3439.69,0


In [16]:
df_compras_base['dia'] = df_compras_base['show'].map(df_shows.set_index('show')['dia']).fillna(0)

In [17]:
df_compras_base

,nome,show,gastos,dia
0,Joseph Wray,Symbolic Toy,3142.35,3
1,Barbara Mcneil,Symbolic Toy,2431.78,3
2,Eduardo Espenshade,Symbolic Toy,386.32,3
3,Lucia Hayes,Symbolic Toy,3373.49,3
4,Johnny Hairston,Symbolic Toy,2805.88,3
...,...,...,...,...
495,Cassandra Gonsalves,Terrestrial Chair,154.39,1
496,Christopher Mueller,Symbolic Toy,1797.44,3
497,Christopher Mueller,Nascent Letter,1511.91,2
498,Christopher Mueller,Nascent Letter,3439.69,2


Consolidação de linhas duplicadas, somando os gastos



In [18]:
df_compras = df_compras_base.groupby(['nome', 'show', 'dia'],as_index=False)['gastos'].sum()
df_compras

,nome,show,dia,gastos
0,Adelina Thews,Nascent Letter,2,371.39
1,Adelina Thews,Terrestrial Chair,1,2372.34
2,Alan Gravois,Nascent Letter,2,463.15
3,Albert Fadden,Terrestrial Chair,1,1637.19
4,Alberto Gumm,Terrestrial Chair,1,2297.47
...,...,...,...,...
324,William Polite,Terrestrial Chair,1,304.31
325,William Swann,Symbolic Toy,3,6408.16
326,William Swann,Terrestrial Chair,1,2962.95
327,Willie Rodriguez,Symbolic Toy,3,1924.87


merge DF's

In [19]:

df_bckp_merge =df_ingressos.merge(df_compras, on=['nome', 'dia', 'show'], how='outer', indicator=True)
df_bckp_completo= df_bckp_merge.sort_values(by=['nome'])
df_bckp_completo.reset_index(drop=True, inplace=True)
df_bckp_completo

,ano,dia,mes,nome,status,tipo,show,gastos,_merge
0,NaN,2,NaN,Adelina Thews,NaN,NaN,Nascent Letter,371.39,right_only
1,NaN,1,NaN,Adelina Thews,NaN,NaN,Terrestrial Chair,2372.34,right_only
2,NaN,2,NaN,Alan Gravois,NaN,NaN,Nascent Letter,463.15,right_only
3,NaN,1,NaN,Albert Fadden,NaN,NaN,Terrestrial Chair,1637.19,right_only
4,NaN,1,NaN,Alberto Gumm,NaN,NaN,Terrestrial Chair,2297.47,right_only
...,...,...,...,...,...,...,...,...,...
367,NaN,1,NaN,William Swann,NaN,NaN,Terrestrial Chair,2962.95,right_only
368,NaN,3,NaN,William Swann,NaN,NaN,Symbolic Toy,6408.16,right_only
369,NaN,3,NaN,Willie Rodriguez,NaN,NaN,Symbolic Toy,1924.87,right_only
370,2020.0,3,1.0,Yong Baltazar,Nao Concluido,VIP,Symbolic Toy,NaN,left_only


#Back-up 

In [20]:
df_completo = df_bckp_completo

In [21]:
df_completo.drop(["ano","mes", "_merge"], axis=1, inplace=True)
df_completo

,dia,nome,status,tipo,show,gastos
0,2,Adelina Thews,NaN,NaN,Nascent Letter,371.39
1,1,Adelina Thews,NaN,NaN,Terrestrial Chair,2372.34
2,2,Alan Gravois,NaN,NaN,Nascent Letter,463.15
3,1,Albert Fadden,NaN,NaN,Terrestrial Chair,1637.19
4,1,Alberto Gumm,NaN,NaN,Terrestrial Chair,2297.47
...,...,...,...,...,...,...
367,1,William Swann,NaN,NaN,Terrestrial Chair,2962.95
368,3,William Swann,NaN,NaN,Symbolic Toy,6408.16
369,3,Willie Rodriguez,NaN,NaN,Symbolic Toy,1924.87
370,3,Yong Baltazar,Nao Concluido,VIP,Symbolic Toy,NaN


# 1. Qual a média de gastos de pessoas com ingresso Pista?

criando filtro para o ingresso de pista do dataframe completo

In [22]:
df_mask=df_completo['tipo']=='Pista'


In [102]:
df_pista = df_completo[df_mask]

In [ ]:
#df_pista

Limpando os NaN e os duplicados que estão contidos no DF

In [104]:
df_pista = df_pista.dropna().drop_duplicates(subset=['dia','nome'])
df_pista

,dia,nome,status,tipo,show,gastos
13,1,Andrea Galyen,Nao Concluido,Pista,Terrestrial Chair,4304.86
74,1,Daniel Puckett,Nao Concluido,Pista,Terrestrial Chair,1489.02
84,1,Debra Howard,Problema no Pagamento,Pista,Terrestrial Chair,5879.15
94,1,Donna Wal,Concluido,Pista,Terrestrial Chair,2841.61
112,2,Emil Nord,Nao Concluido,Pista,Nascent Letter,4798.81
139,1,Gretchen Hinojosa,Nao Concluido,Pista,Terrestrial Chair,2867.60
154,2,James Reaves,Problema no Pagamento,Pista,Nascent Letter,5035.27
159,1,Jarrod Adler,Nao Concluido,Pista,Terrestrial Chair,1070.84
180,2,John Rich,Problema no Pagamento,Pista,Nascent Letter,4739.62
196,1,Joshua Sanderson,Nao Concluido,Pista,Terrestrial Chair,1775.60


calculando a média dos gastos

In [105]:
media_gastos_pista = df_pista['gastos'].mean()

In [106]:
print ("A média de gastos de pessoas com ingresso Pista foi de ${:.2f}".format(media_gastos_pista))

A média de gastos de pessoas com ingresso Pista foi de $3781.83


#2. Quais pessoas não compareceram aos shows?


Como houve ambuigidade na pergunta estou disponibilizando as duas formas:


*   Todas as pessoas que não compareceram aos shows.
*   Todas as pessoas que compraram ingresso e não compareceram aos shows.



**A) todas as pessoas que não compareceram aos shows.**

In [82]:
df_ausente_bckp = df_completo.loc[df_completo.gastos.isnull()]
df_ausente =df_ausente_bckp
#df_ausente.style.hide_index()

Removendo as colunas e os duplicados.

In [84]:
df_ausente= df_ausente.drop(columns=["dia", "status","tipo", "show", "gastos"]).drop_duplicates()

Lista das pessoas

In [107]:
 df_ausente.style.hide_index()

nome
Alice Gamba
Amy Mejia
Brenda Key
Brenda Kozak
Courtney Edwards
Curtis Hernandez
Deborah Silsby
Donna Chandler
Edward Smith
Elizabeth Daly


**B)Todas as pessoas que compraram ingresso e não compareceram aos shows.**

fazendo a máscara dos pedidos concluídos.

In [78]:
df_filtro_concluido=df_ausente_bckp['status']=="Concluido"
df_ausente_concluido = df_ausente[df_filtro_concluido]

Removendo colunas indesejadas e removendo os duplicados.

In [80]:
df_ausente_concluido =df_ausente_concluido.drop(columns=["dia", "status","tipo", "show", "gastos"]).drop_duplicates()

Lista das pessoas

In [81]:
df_ausente_concluido.style.hide_index()

nome
Alice Gamba
Brenda Key
Courtney Edwards
Deborah Silsby
Frances Lambert
Linda Batiz
Marcela Nelson
Miguel Ramey
Pamela Nixon
Ricardo Clasby


#3. Quais pessoas compraram ingressos com concorrentes?


Listando quais campos tem o NaN

In [33]:
df_completo.isna().sum()

dia         0
nome        0
status    272
tipo      272
show        0
gastos     33
dtype: int64

Criando filtro para que mostre apenas as pessoas que tiveram status do ingresso diferente de concluido

In [34]:
df_mask_ingressos_idx= df_completo['status']!='Concluido'
filtro_df_ingressos = df_completo[df_mask_ingressos_idx]
filtro_df_ingressos


,dia,nome,status,tipo,show,gastos
0,2,Adelina Thews,NaN,NaN,Nascent Letter,371.39
1,1,Adelina Thews,NaN,NaN,Terrestrial Chair,2372.34
2,2,Alan Gravois,NaN,NaN,Nascent Letter,463.15
3,1,Albert Fadden,NaN,NaN,Terrestrial Chair,1637.19
4,1,Alberto Gumm,NaN,NaN,Terrestrial Chair,2297.47
...,...,...,...,...,...,...
366,1,William Polite,NaN,NaN,Terrestrial Chair,304.31
367,1,William Swann,NaN,NaN,Terrestrial Chair,2962.95
368,3,William Swann,NaN,NaN,Symbolic Toy,6408.16
369,3,Willie Rodriguez,NaN,NaN,Symbolic Toy,1924.87


Removendo as pessoas que não compareceram aos shows.

In [73]:
concorrentes_status = filtro_df_ingressos.dropna(subset=['gastos'])

Removendo colunas indesejadas e removendo os duplicados.

In [75]:
concorrentes_status =concorrentes_status.drop(columns=["status","tipo", "gastos"]).drop_duplicates()

**Lista de pessoas que compraram ingressos no concorrentes**

In [76]:
concorrentes_status.style.hide_index()

dia,nome,show
2,Adelina Thews,Nascent Letter
1,Adelina Thews,Terrestrial Chair
2,Alan Gravois,Nascent Letter
1,Albert Fadden,Terrestrial Chair
1,Alberto Gumm,Terrestrial Chair
3,Allen Bylsma,Symbolic Toy
1,Allen Bylsma,Terrestrial Chair
2,Allison Magill,Nascent Letter
2,Amanda Jones,Nascent Letter
3,Andrea Galyen,Symbolic Toy


#Qual o dia com maior gasto?

In [108]:
df_dia_gastos = df_completo

Removendo os duplicados.

In [ ]:
df_dia_gastos =df_dia_gastos.drop_duplicates(subset=['gastos', 'dia'])

Fazendo a combinação dos dias e tirando a média

In [70]:
df_dia_gastos.groupby(['dia']).mean()

,gastos
dia,
1,2711.273814
2,2544.961132
3,3162.365865


In [71]:
print("O dia de maior gasto no festival foi o dia 3")

O dia de maior gasto no festival foi o dia 3


#5. Faça uma lista com os clientes que desistiram de comprar o ingresso com a AT, a soma do valor que foi gasto durante os shows e quais shows eles desistiram de comprar.

Filtro tirando as pessoas que não foram ao show e das pessoas que nunca procuraram a AT para comprar ingresso.

In [128]:
desistencia_df_base =df_completo.dropna(subset=['gastos', 'status'])
desistencia_df_base =desistencia_df_base.rename(columns={'show': 'shows'})


Aplicando máscara para tirar os que compraram o ingresso com a AT.

In [129]:
mask_concluido =(desistencia_df_base['status'] == 'Concluido')
desistencia_df = desistencia_df_base.loc[~mask_concluido]

Removendo colunas indesejadas

In [130]:
desistencia_df =desistencia_df.drop(columns=["dia", "status","tipo"]).drop_duplicates()


Corrigindo ordem das colunas.

In [131]:
desistencia_df = desistencia_df[['nome','gastos','shows']]

Função para o groupby somar os gastos e concatenar os shows

In [140]:

def f(x):
     return Series(dict(gastos = x['gastos'].sum(), shows = "{%s}" % ', '.join(x['shows'])))

Aplicação do groupby

In [141]:
desistencia_df =desistencia_df.groupby('nome').apply(f)

In [142]:
desistencia_df = desistencia_df.reset_index(level=0)


In [143]:
desistencia_df = desistencia_df.infer_objects()
desistencia_df.dtypes

nome       object
gastos    float64
shows      object
dtype: object

In [154]:
js = desistencia_df.to_json(double_precision = 2,default_handler = callable,orient = 'records')

In [155]:
js

'[{"nome":"Andrea Galyen","gastos":5133.81,"shows":"{Symbolic Toy, Terrestrial Chair}"},{"nome":"Benjamin Mcdonald","gastos":2309.81,"shows":"{Terrestrial Chair}"},{"nome":"Blanche Knox","gastos":924.31,"shows":"{Terrestrial Chair}"},{"nome":"Charles Hershenson","gastos":163.44,"shows":"{Terrestrial Chair}"},{"nome":"Daniel Puckett","gastos":1489.02,"shows":"{Terrestrial Chair}"},{"nome":"Debra Howard","gastos":5879.15,"shows":"{Terrestrial Chair}"},{"nome":"Donna Wal","gastos":3279.79,"shows":"{Symbolic Toy}"},{"nome":"Edna Martinez","gastos":9415.36,"shows":"{Nascent Letter, Terrestrial Chair}"},{"nome":"Emil Nord","gastos":4798.81,"shows":"{Nascent Letter}"},{"nome":"Frank Davilla","gastos":1724.35,"shows":"{Nascent Letter}"},{"nome":"Frederick Bank","gastos":11883.25,"shows":"{Terrestrial Chair}"},{"nome":"Gerald Allen","gastos":3447.17,"shows":"{Terrestrial Chair}"},{"nome":"Gretchen Hinojosa","gastos":2867.6,"shows":"{Terrestrial Chair}"},{"nome":"James Reaves","gastos":5035.27,"

In [151]:
data_dict = json.loads(js)

In [152]:
data_dict

[['Andrea Galyen', 5133.81, '{Symbolic Toy, Terrestrial Chair}'],
 ['Benjamin Mcdonald', 2309.81, '{Terrestrial Chair}'],
 ['Blanche Knox', 924.31, '{Terrestrial Chair}'],
 ['Charles Hershenson', 163.44, '{Terrestrial Chair}'],
 ['Daniel Puckett', 1489.02, '{Terrestrial Chair}'],
 ['Debra Howard', 5879.15, '{Terrestrial Chair}'],
 ['Donna Wal', 3279.79, '{Symbolic Toy}'],
 ['Edna Martinez', 9415.36, '{Nascent Letter, Terrestrial Chair}'],
 ['Emil Nord', 4798.81, '{Nascent Letter}'],
 ['Frank Davilla', 1724.35, '{Nascent Letter}'],
 ['Frederick Bank', 11883.25, '{Terrestrial Chair}'],
 ['Gerald Allen', 3447.17, '{Terrestrial Chair}'],
 ['Gretchen Hinojosa', 2867.6, '{Terrestrial Chair}'],
 ['James Reaves', 5035.27, '{Nascent Letter}'],
 ['Jarrod Adler', 1070.84, '{Terrestrial Chair}'],
 ['Jeffrey Pace', 5810.78, '{Terrestrial Chair}'],
 ['John Rich', 7505.78, '{Symbolic Toy, Nascent Letter}'],
 ['Joseph Wray', 3142.35, '{Symbolic Toy}'],
 ['Joshua Sanderson', 1775.6, '{Terrestrial Chair